In [ ]:
# # -*- coding: utf-8 -*-
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import os
# from datetime import datetime
# from matplotlib.colors import LinearSegmentedColormap
# import warnings
# warnings.filterwarnings("ignore")

# # ======================================================================
# # تنظیمات
# # ======================================================================
# INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
# OUTPUT_FOLDER = r"C:\BI\Correlation_Heatmap_Monitoring"
# os.makedirs(OUTPUT_FOLDER, exist_ok=True)
# LOG_FILE = os.path.join(OUTPUT_FOLDER, "Log.txt")

# WINDOW_SIZE = 300
# STEP = 20
# CHANGE_THRESHOLD = 0.30

# def log(msg):
#     print(msg)
#     with open(LOG_FILE, "a", encoding="utf-8") as f:
#         f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

# log("شروع مانیتورینگ لحظه‌ای کوریلیشن - نسخه نهایی و کاملاً بدون خطا")

# # ======================================================================
# # خواندن داده
# # ======================================================================
# df = pd.read_excel(INPUT_FILE)
# df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
# df = df.sort_values("datetime").reset_index(drop=True)

# friendly_names = {
#     "AssetID_8341": "Filter DP",
#     "AssetID_8342": "Tank Level",
#     "AssetID_8343": "Tank Temp",
#     "AssetID_8344": "Vapor Press",
#     "AssetID_8346": "Overspeed Press",
#     "AssetID_9286": "Main Pump Press",
#     "AssetID_9287": "Line Pressure",
# }
# df.rename(columns=friendly_names, inplace=True)
# sensor_cols = list(friendly_names.values())

# df = df[["datetime"] + sensor_cols].dropna().reset_index(drop=True)
# log(f"داده آماده: {len(df)} رکورد")

# # ======================================================================
# # 1. هیت‌مپ مرجع
# # ======================================================================
# cmap = LinearSegmentedColormap.from_list("my_cmap", ["#0066CC", "#FFFFFF", "#CC0033"])

# plt.figure(figsize=(10, 8))
# sns.heatmap(df[sensor_cols].corr(), annot=True, cmap=cmap, center=0, vmin=-1, vmax=1,
#             square=True, fmt=".2f", linewidths=0.5, cbar_kws={"shrink": 0.8})
# plt.title("Reference Correlation Heatmap (Full Dataset)", fontsize=14, pad=20)
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.tight_layout()
# plt.savefig(os.path.join(OUTPUT_FOLDER, "00_Reference.png"), dpi=300, bbox_inches='tight')
# plt.close()
# log("هیت‌مپ مرجع ذخیره شد")

# # ======================================================================
# # 2. محاسبه کوریلیشن متحرک + ساخت هیت‌مپ + ذخیره جفت‌ها
# # ======================================================================
# all_correlations = []

# for i in range(WINDOW_SIZE, len(df), STEP):
#     window = df.iloc[i - WINDOW_SIZE:i]
#     timestamp = window["datetime"].iloc[-1]
#     corr_mat = window[sensor_cols].corr()

#     # تبدیل ماتریس به جفت‌های سنسور
#     for s1 in sensor_cols:
#         for s2 in sensor_cols:
#             if s1 < s2:  # فقط جفت‌های یک‌بار (A,B) نه (B,A)
#                 corr_val = corr_mat.loc[s1, s2]
#                 all_correlations.append({
#                     "Timestamp": timestamp,
#                     "Sensor_A": s1,
#                     "Sensor_B": s2,
#                     "Correlation": corr_val
#                 })

#     # ساخت هیت‌مپ هر 50 مرحله
#     if (len(all_correlations) // 21) % 50 == 0 or i >= len(df) - STEP:
#         plt.figure(figsize=(10, 8))
#         sns.heatmap(corr_mat, annot=True, cmap=cmap, center=0, vmin=-1, vmax=1,
#                     square=True, fmt=".2f", linewidths=0.5)
#         plt.title(f"Rolling Correlation\n{timestamp.strftime('%Y-%m-%d %H:%M')}", fontsize=13)
#         plt.xticks(rotation=45, ha='right')
#         plt.yticks(rotation=0)
#         plt.tight_layout()
#         fname = f"Rolling_{i//STEP:04d}_{timestamp.strftime('%Y%m%d_%H%M')}.png"
#         plt.savefig(os.path.join(OUTPUT_FOLDER, fname), dpi=300, bbox_inches='tight')
#         plt.close()

# # تبدیل به دیتافریم
# rolling_df = pd.DataFrame(all_correlations)
# rolling_df = rolling_df.sort_values("Timestamp")
# log(f"تعداد جفت کوریلیشن محاسبه شده: {len(rolling_df)}")

# # ذخیره داده خام
# rolling_df.to_csv(os.path.join(OUTPUT_FOLDER, "Rolling_Correlation_Values.csv"), 
#                   index=False, encoding="utf-8-sig")

# # ======================================================================
# # 3. تشخیص تغییرات ناگهانی (هشدار هوشمند) — بدون هیچ خطایی!
# # ======================================================================
# alerts = []

# # برای هر جفت سنسور، تغییرات را محاسبه کن
# for (s1, s2), group in rolling_df.groupby(["Sensor_A", "Sensor_B"]):
#     group = group.sort_values("Timestamp").reset_index(drop=True)
#     group["Change"] = group["Correlation"].diff().abs()
#     big_changes = group[group["Change"] > CHANGE_THRESHOLD]
    
#     for _, row in big_changes.iterrows():
#         alerts.append({
#             "Timestamp": row["Timestamp"],
#             "Sensor_A": s1,
#             "Sensor_B": s2,
#             "Previous_Correlation": group.loc[row.name - 1, "Correlation"] if row.name > 0 else np.nan,
#             "Current_Correlation": row["Correlation"],
#             "Correlation_Change": row["Change"]
#         })

# alerts_df = pd.DataFrame(alerts)
# if not alerts_df.empty:
#     alerts_df = alerts_df.sort_values("Correlation_Change", ascending=False)

# alert_path = os.path.join(OUTPUT_FOLDER, "Significant_Correlation_Changes.csv")
# alerts_df.to_csv(alert_path, index=False, encoding="utf-8-sig")

# log(f"تعداد هشدارهای مهم: {len(alerts_df)}")
# if len(alerts_df) > 0:
#     print(f"\n{len(alerts_df)} تغییر ناگهانی مهم تشخیص داده شد!")
#     print(alerts_df.head(10)[["Timestamp", "Sensor_A", "Sensor_B", 
#                               "Previous_Correlation", "Current_Correlation", "Correlation_Change"]].round(4))
# else:
#     print("\nهیچ تغییر ناگهانی مهمی در الگوی همبستگی تشخیص داده نشد — سیستم پایدار است")

# # ======================================================================
# # 4. ساخت گیف
# # ======================================================================
# try:
#     import imageio.v3 as imageio
#     images = []
#     for f in sorted([f for f in os.listdir(OUTPUT_FOLDER) if f.startswith("Rolling_") and f.endswith(".png")]):
#         images.append(imageio.imread(os.path.join(OUTPUT_FOLDER, f)))
#     if images:
#         gif_path = os.path.join(OUTPUT_FOLDER, "Correlation_Evolution.gif")
#         imageio.mimsave(gif_path, images, duration=800, loop=0)
#         print(f"گیف ساخته شد: {gif_path}")
# except:
#     print("گیف ساخته نشد (اختیاری)")

# # ======================================================================
# # پایان
# # ======================================================================
# print("\n" + "="*100)
# print("همه چیز با موفقیت انجام شد!")
# print(f"خروجی‌ها در: {OUTPUT_FOLDER}")
# print("="*100)
# print("   • 00_Reference.png")
# print("   • Rolling_*.png (هیت‌مپ‌های پویا)")
# print("   • Rolling_Correlation_Values.csv")
# print("   • Significant_Correlation_Changes.csv ← هشدارهای هوشمند!")
# print("   • Correlation_Evolution.gif")
# print("="*100)
# log("پایان موفق")

شروع مانیتورینگ لحظه‌ای کوریلیشن - نسخه نهایی و کاملاً بدون خطا
داده آماده: 10925 رکورد
هیت‌مپ مرجع ذخیره شد
تعداد جفت کوریلیشن محاسبه شده: 11172
تعداد هشدارهای مهم: 131

131 تغییر ناگهانی مهم تشخیص داده شد!
              Timestamp         Sensor_A         Sensor_B  \
97  2023-02-17 06:05:48  Overspeed Press       Tank Level   
98  2023-04-08 21:05:50  Overspeed Press       Tank Level   
35  2023-04-08 21:05:50    Line Pressure  Overspeed Press   
64  2022-07-02 05:39:57  Main Pump Press  Overspeed Press   
7   2023-02-17 06:05:48        Filter DP  Overspeed Press   
105 2023-04-08 21:05:50  Overspeed Press        Tank Temp   
104 2023-02-17 06:05:48  Overspeed Press        Tank Temp   
117 2023-02-17 06:05:48       Tank Level        Tank Temp   
130 2023-04-08 21:05:50        Tank Temp      Vapor Press   
84  2023-04-08 21:05:50  Main Pump Press        Tank Temp   

     Previous_Correlation  Current_Correlation  Correlation_Change  
97                -0.5317               0.9696     

In [7]:
# -*- coding: utf-8 -*-
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
from matplotlib.colors import LinearSegmentedColormap

# ======================================================================
# تنظیمات
# ======================================================================
INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
OUTPUT_FOLDER = r"C:\BI\Correlation_Heatmap_Monitoring"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# نام‌گذاری سنسورها (کوتاه و حرفه‌ای)
friendly_names = {
    "AssetID_8341": "Filter DP",
    "AssetID_8342": "Tank Level",
    "AssetID_8343": "Tank Temp",
    "AssetID_8344": "Vapor Press",
    "AssetID_8346": "Overspeed Press",
    "AssetID_9286": "Main Pump Press",
    "AssetID_9287": "Line Pressure",
}

# رنگ‌بندی حرفه‌ای
cmap = LinearSegmentedColormap.from_list("RdBu_div", ["#0066CC", "#FFFFFF", "#CC0033"])

print("Reading data...")
df = pd.read_excel(INPUT_FILE)

# ساخت ستون datetime
df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
df = df.dropna(subset=["datetime"]).sort_values("datetime").reset_index(drop=True)
df.rename(columns=friendly_names, inplace=True)
sensor_cols = list(friendly_names.values())

latest_time = df["datetime"].max()
print(f"Latest record: {latest_time.strftime('%Y-%m-%d %H:%M')}")

# تعریف سه بازه زمانی
heatmaps = [
    {
        "filename": "heatmap1_recent.png",
        "title": "Correlation Heatmap - Last 30 Days (Most Recent)",
        "start_days_ago": 0,
        "end_days_ago": 30
    },
    {
        "filename": "heatmap2_previous.png",
        "title": "Correlation Heatmap - 30 to 60 Days Ago",
        "start_days_ago": 30,
        "end_days_ago": 60
    },
    {
        "filename": "heatmap3_older.png",
        "title": "Correlation Heatmap - 60 to 90 Days Ago",
        "start_days_ago": 60,
        "end_days_ago": 90
    },
]

plt.figure(figsize=(10.5, 8.5))

for hm in heatmaps:
    start_date = latest_time - timedelta(days=hm["end_days_ago"])
    end_date   = latest_time - timedelta(days=hm["start_days_ago"])
    
    mask = (df["datetime"] >= start_date) & (df["datetime"] < end_date)
    period_df = df[mask]
    
    if len(period_df) < 30:
        print(f"Not enough data for {hm['filename']} → only {len(period_df)} records")
        continue
    
    corr = period_df[sensor_cols].corr()
    
    plt.clf()
    sns.heatmap(corr, annot=True, cmap=cmap, center=0, vmin=-1, vmax=1,
                square=True, fmt=".2f", linewidths=0.8, cbar_kws={"shrink": 0.8},
                xticklabels=sensor_cols, yticklabels=sensor_cols)
    
    plt.title(f"{hm['title']}\n"
              f"{start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} | "
              f"N = {len(period_df):,} records",
              fontsize=14, pad=25, fontweight='bold')
    
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout()
    
    output_path = os.path.join(OUTPUT_FOLDER, hm["filename"])
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Saved → {hm['filename']} ({len(period_df):,} records)")

plt.close()

print("\n" + "="*85)
print("All done! Three professional English-titled heatmaps created:")
print("   • heatmap1_recent.png   → Last 30 days")
print("   • heatmap2_previous.png → 30–60 days ago")
print("   • heatmap3_older.png    → 60–90 days ago")
print(f"Location: {OUTPUT_FOLDER}")
print("="*85)

Reading data...
Latest record: 2025-11-17 04:10
Saved → heatmap1_recent.png (179 records)
Saved → heatmap2_previous.png (180 records)
Saved → heatmap3_older.png (159 records)

All done! Three professional English-titled heatmaps created:
   • heatmap1_recent.png   → Last 30 days
   • heatmap2_previous.png → 30–60 days ago
   • heatmap3_older.png    → 60–90 days ago
Location: C:\BI\Correlation_Heatmap_Monitoring
